In [1]:
import sys
sys.path.append("../")

from PIL import Image

import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

from relic import ReLIC
from encoders import resnet18
from aug import get_relic_aug_inference, get_relic_aug

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = get_relic_aug_inference()

train_ds = torchvision.datasets.STL10("../data",
                                  split='train',
                                  transform=transform,
                                  download=True)
val_ds = torchvision.datasets.STL10("../data",
                                  split='test',
                                  transform=transform,
                                  download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
len(train_ds), len(val_ds)

(5000, 8000)

In [5]:
ckpt = torch.load("../models/relic_model.pth")

encoder = resnet18()
model = ReLIC(encoder)
model.load_state_dict(ckpt)

model = model.eval().to(device)

In [6]:
from tqdm.auto import tqdm

def get_embs_targets(ds):
    idx = 0
    embs, targets = [], []
    for idx, (image, target) in enumerate(tqdm(ds)):
        with torch.no_grad():
            image = image.unsqueeze(0).to(device)
            out = model.get_features(image)
            features = out[0].cpu().detach().tolist()   
            embs.append(features)
            targets.append(target)
    return embs, targets

In [7]:
embs, targets = get_embs_targets(train_ds)
embs_val, targets_val = get_embs_targets(val_ds)

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

In [8]:
import numpy as np

embeddings = np.array(embs)
labels = np.array(targets)
embeddings_val = np.array(embs_val)
labels_val = np.array(targets_val)

In [9]:
print(embeddings.shape)
print(labels.shape)
print(embeddings_val.shape)
print(labels_val.shape)

(5000, 512)
(5000,)
(8000, 512)
(8000,)


In [10]:
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
 
X_train, X_test = embeddings, embeddings_val
y_train, y_test = labels, labels_val

print("train", X_train.shape[0], len(y_train))
print("test", X_test.shape[0], len(y_test))
 
clf = LogisticRegression(max_iter=100)
clf = CalibratedClassifierCV(clf)
 
clf.fit(X_train, y_train)
 
y_pred = clf.predict(X_test)
 
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
 
print("Accuracy: ", acc)
print("Confusion matrix: \n", conf_matrix)
print("Classification report: \n", class_report)
 
y_pred_train = clf.predict(X_train)
class_report = classification_report(y_train, y_pred_train)
print("Classification report train: \n", class_report)

train 5000 5000
test 8000 8000


/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Accuracy:  0.705125
Confusion matrix: 
 [[657  43  14   6   2   7   7   2  44  18]
 [ 18 580   0  71  18  30  10  70   3   0]
 [ 30   2 691   8   0   2   9   2  12  44]
 [  4  77   1 459  58  80  20  91   6   4]
 [  2  43   1  69 531  33  85  32   2   2]
 [  2  73   1 136  51 303 100 132   0   2]
 [  5   8   2  15  32  54 638  38   3   5]
 [  2  95   0  76  27  60  27 509   3   1]
 [ 47   3  12   7   0   3   0   1 696  31]
 [ 30   3  79  14   1   5  13   3  75 577]]
Classification report: 
               precision    recall  f1-score   support

           0       0.82      0.82      0.82       800
           1       0.63      0.72      0.67       800
           2       0.86      0.86      0.86       800
           3       0.53      0.57      0.55       800
           4       0.74      0.66      0.70       800
           5       0.53      0.38      0.44       800
           6       0.70      0.80      0.75       800
           7       0.58      0.64      0.61       800
           8     